In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
sns.set(style="white")


import plotly
import plotly.express as px

import geopandas as gpd

from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)


df = pd.read_csv(
    
    'cities_internet_prices_historical.24-10-2021.csv'
)

ModuleNotFoundError: No module named 'geopandas'

Basic informations:

* 695 rows
* 14 columns

This dataset contatins data about internet prices from whole world in years 2010 - 2020.
I' m going to use this dataset to answer questions I'm interested in:

* WHERE INTERNET WAS THE CHEAPEST AND WHERE IT WAS THE MOST EXPENSIVE?
* HOW INTERNET PRICES DISTRIBUTION CHANGED OVER YEARS?
* HOW INTERNET PRICES WAS CHANGING OVER THE YEARS IN POLAND?


## 1.DATA PREPROCESSING

First five and last five values:

In [ ]:
df.head()

In [ ]:
df.tail()

First thing which is visible for me is that in many columns we can see value 0. it may be a big problem which may make all further analysis unreliable

Null values?

In [ ]:
df.isnull().sum()

there are supposed to be 90 empty values in Region but it's not right, it's much much more and not only in this column.


Let's replace 0. with Nan.

In [ ]:
df.replace(0., np.nan, inplace=True)
df.isnull().sum()

Before analysis go any further I'm going to replace columns names.

In [ ]:
new_columns = list(df.columns[:3])
for column in df.columns[3:]:
    new_columns.append('price_in_' + column[-4:])

df.columns = new_columns

In [ ]:
df.sample()

Now we can see how much null vales we have in real, let's see how it looks in percentages

In [ ]:
from typing import List

def percentage_nan_numeric(data_frame=df):
    """This function takes pandas DataFrame by default it's df,
    it returns percentile of null values in every numeric column"""
    
    numeric_columns = df.select_dtypes(include=[np.number])
    
    total_rows:int = numeric_columns.shape[0]
    
    numeric_columns_null_values:List[int] = numeric_columns.isnull().sum().to_dict()
    
    for column, null_values in numeric_columns_null_values.items():
        print(f'Percentage of null values {column} : {round((null_values / total_rows) * 100)}%' )
    

In [ ]:
percentage_nan_numeric()

There's many null values in this column, but we can see pattern, with every year it's less and less.

But there's one outlier - Internet price in 2020 has 32 %, it doesn't match the pattern I described above.

Anyway I'll use one method to handle with null values.

* I will take mean of not null values from every column and replace null values with this mean.

* good approach migth be using supervised method KNN from sklearn. More about KNN may be find in this great article: [KNN](https://towardsdatascience.com/machine-learning-basics-with-the-k-nearest-neighbors-algorithm-6a6e71d01761) but I'll leave it only as a clue because I'm not going to use it

### First approach - means

In [ ]:
def inpute_means(data_frame=df):
    
    """takes one kwarg data_frame by default it's df 
    returns new data frame with null values filled with means"""
    
    not_numeric = df.select_dtypes(exclude=[np.number])
    numeric = df.select_dtypes(include=[np.number])
    
    for column in numeric:
        numeric[column].fillna(np.mean(df[column]), inplace=True)
        
    return pd.concat([not_numeric, numeric], axis=1) # connecting not numeric coulumns with numeric

In [ ]:
df_copy = df # setting the copy of the old data frame
df = inpute_means()

Basics informations:

In [ ]:
df.describe()

[IQR range](https://www.statisticshowto.com/probability-and-statistics/interquartile-range/) - looks really bad in prices from 2010 to 2015,
[standard deviation](https://en.wikipedia.org/wiki/Standard_deviation)
is also quite alarming but this is the result of what was done before (inputing means). Droping some columns might be inevitable.

## 2. DATA VISUALIZATIONS

In [ ]:
def plot_histogram(column_name, data_frame=df):
    
    """arguments: column_name:str - name of the column to be ploted
       returns: histogram object
    """
    
    #  in case when column is not in data frame
    if column_name not in data_frame.columns:
        raise ValueError(f'Chose correct column from data frame colums: {data_frame.columns}')
        
    fig = px.histogram(data_frame=data_frame.sort_values(by=column_name), x=column_name,
                       color_discrete_sequence=['blue'])
    
    
    fig.update_layout(font=dict(family='Lato', size=16), 
                      title=dict(text=f'<b>histogram - {column_name}<b>',
                                font=dict(size=24),
                                x=.5),
                     plot_bgcolor='lightblue',
                     paper_bgcolor='lightblue',
                     xaxis=dict(showgrid=False),
                     yaxis=dict(showgrid=False))
    
    fig.show()

In [ ]:
numeric_columns = df.select_dtypes(include=[np.number])

for column in numeric_columns:
    plot_histogram(column)

At the first glance:

I think that droping prices from 2010 to 2015 (inclusive) is 
necessarily

In [ ]:
columns_to_drop = []

for x in range(2010, 2016):
    column_name:str = 'price_in_' + str(x)
    columns_to_drop.append(column_name)
    
columns_to_drop

In [ ]:
df.drop(columns=columns_to_drop, inplace=True)

After handling with null values it's time to handle with outliers, i'm going to set upper bound as less than 80.

In [ ]:
internet_prices = (
       df[
    (df['price_in_2016'] < 80) 
    & (df['price_in_2017'] < 80)
    & (df['price_in_2018'] < 80) 
    & (df['price_in_2019'] < 80) 
    & (df['price_in_2020'] < 80)]
   )

In [ ]:
numeric_columns = internet_prices.select_dtypes(include=[np.number])

for column in numeric_columns:
    plot_histogram(column, data_frame=numeric_columns)

### WHERE INTERNET WAS THE CHEAPEST AND WHERE IT WAS THE MOST EXPENSIVE?

In [ ]:
def bar_plot(column_name, data_frame=df, tribe='value_counts', by=None, limit=3, 
             ascending=False,**kwargs):
    
    """arguments:
    column_name:str name of column from pandas data drame
    data_frame:pandas data frame default df
    tribe:str value counts by default, change to 'sort' if you want to have
    values in decreasing order
    by:str works only if sort tribe chosed, takes name of the column
        you want to sort values by"
    limit:int limit of values displayed"""
    
    
    # if column is not str type or column not in columns of data frame
    if type(column_name) != str or column_name not in data_frame.columns:
        raise ValueError('Incorect column name or type')
        
    if type(limit) != int:
        raise ValueErroc(f'excepted int type, got {type(limit)}')
    
    
    # sorted data frame
    if tribe == 'sort' :
        # if column is not in data frame
        if not by or by not in data_frame.columns:
            raise ValueError('by paramter must be column from data frame!')
        # in case if ascending is not provided as boolean
        if type(ascending) != bool:
            raise ValueError(f'expected ascending paramter to be bool type got: {ascending}')
        data = data_frame.sort_values(by=by, ascending=ascending).head(limit)
        y = data[by].values
        x = data[column_name]
        title = by
    
    # value counts
    elif tribe == 'value_counts':
        data = data_frame[column_name].value_counts().head(limit)
        x=data.index
        y=data.values
        title = column_name
    
    
    fig = px.bar(x=x, 
                 y=y,
            color_discrete_sequence=['mintcream']
                )

    fig.update_layout(font=dict(family='Lato', size=16), 
                  title=dict(text=f'<b>{title}<b>',
                           font=dict(size=30), x=.5), 
                  paper_bgcolor= 'lightblue', plot_bgcolor='lightblue',
                 xaxis=dict(title=f'', showgrid=False),
                 yaxis=dict(title=f'count', showgrid=False))
        
    fig.show()

### 2016

#### CHEAPEST

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2016',
        data_frame=internet_prices, limit=10, ascending=True)

#### MOST EXPENSIVE

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2016',
        data_frame=internet_prices, limit=10)

### 2017

#### CHEAPEST

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2017',
        data_frame=internet_prices, limit=10, ascending=True)

#### MOST EXPENSIVE

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2017',
        data_frame=internet_prices, limit=10, ascending=False)

### 2018

### CHEAPEST

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2018',
        data_frame=internet_prices, limit=10, ascending=True)

### MOST EXPENSIVE

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2018',
        data_frame=internet_prices, limit=10)

### 2019

### CHEAPEST

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2019',
        data_frame=internet_prices, limit=10, ascending=True)

### MOST EXPENSIVE

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2019',
        data_frame=internet_prices, limit=10)

### 2020

#### CHEAPEST

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2020',
        data_frame=internet_prices, limit=10, ascending=True)

### MOST EXPENSIVE

In [ ]:
bar_plot(column_name='City', tribe='sort', by='price_in_2020',
        data_frame=internet_prices, limit=10)

### MAP

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

I need to calculate average price for every country first

In [ ]:
averages_for_countries =(internet_prices[['Country', 'price_in_2016', 
                       'price_in_2017', 'price_in_2018',
                       'price_in_2019','price_in_2020']]
                      .groupby('Country').mean())

averages_for_countries

In [ ]:
world.rename(columns={'name': 'Country'}, inplace=True)

world = world.merge(averages_for_countries, how='left', on='Country')

In [ ]:
def map_plot(column_name:str, data_frame=world):
    """this function takes two parameters,
    first is column_name: str and second is data frame 
    by default it's world
    returns: map of the world with values from column_name parameter"""
    fig, ax = plt.subplots(figsize=(15,10))

    ax.set_xticks([]) #  removing ticks
    ax.set_yticks([])

    ax.set_title(column_name, weight = 'bold', fontsize  = 20)
    data_frame.plot(column_name,
           legend = True, 
           ax = ax,
           missing_kwds= {'color': 'lightgrey', 'edgecolor': 'red', 'hatch': '///', 'label': 'No data'},
           cmap='flare', scheme = 'quantiles',
          legend_kwds={'loc': 'lower left'},
)

In [ ]:
columns_to_plot = (
                    'price_in_2016', 'price_in_2017', 'price_in_2018', 
                   'price_in_2019','price_in_2020'
                  )

In [ ]:
for column in columns_to_plot:
    map_plot(column)

Following conclusions can be drawn from the plots above:

* Internet in USA and Canada is relativery expensive, the same situtation is in Australia
* West Europe have relatively lower prices of internet than East Europe
* in last 3 years price of internet in Russia decreased
* Internet in Africa seems to be quite expensive but there's a lot of missing data on this continent
* Internet price in South America seems to decrase over years
* Internet price in Asia seems to dercrease over years as well

Last thing I'm going to see how internet prices changed over year in Poland.

In [ ]:
poland = internet_prices[internet_prices['Country'] == 'Poland']

In [ ]:
columns_to_plot = (
                    'price_in_2016', 'price_in_2017', 'price_in_2018', 
                   'price_in_2019','price_in_2020'
                  )

for column in columns_to_plot:
    bar_plot('City', data_frame=poland, tribe='sort', by=column, limit=11)

I thought that internet price in Warsaw or Krakow will be the biggest,
but according to the plot above - that's not True.